<a href="https://colab.research.google.com/github/geekysilento/Schizophrenia-Detection-CNN/blob/main/Schizophrenia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing and setting up the Dataset from Kaggle

In [ ]:
import os
import time
import numpy as np
import pandas as pd

import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

import warnings
warnings.filterwarnings('ignore')

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

! for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

a was unexpected at this time.


Reading CSV Data


In [ ]:
demographic = pd.read_csv("button-tone-sz/demographic.csv")
demographic

,subject,group,gender,age,education
0,1,0,M,44,16.0
1,2,0,M,39,17.0
2,3,0,M,53,18.0
3,4,0,M,52,15.0
4,5,0,M,41,16.0
...,...,...,...,...,...
76,77,1,M,28,13.0
77,78,1,F,32,16.0
78,79,1,M,37,16.0
79,80,1,M,33,13.0


In [ ]:
columns = pd.read_csv("button-tone-sz/time.csv")
columns

,sample,time_ms
0,1,-1500.0000
1,2,-1499.0234
2,3,-1498.0469
3,4,-1497.0703
4,5,-1496.0938
...,...,...
3067,3068,1495.1172
3068,3069,1496.0938
3069,3070,1497.0703
3070,3071,1498.0469


In [ ]:
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
electrodes_list = list(pd.read_csv("button-tone-sz/columnLabels.csv").columns[4:])

print(diagnosis_dict)


print("Electrodes List \n",electrodes_list)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1}
Electrodes List 
 ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'VEOa'

In [ ]:
def averaged_by_N_rows(a, n):
    """
    A function that averages n rows in a matrix
    """
    shape = a.shape
    assert len(shape) == 2
    assert shape[0] % n == 0
    b = a.reshape(shape[0] // n, n,
                  shape[1])
    mean_vec = b.mean(axis=1)
    return mean_vec

N_AVERAGED = 16
x_counter = 0
X = np.zeros((81 * 100,  9216 * len(electrodes_list) // N_AVERAGED), dtype="float32")
Y = np.zeros(len(X))

part1_path = "button-tone-sz"
part2_path = "button-tone-sz"


column_list = pd.read_csv("button-tone-sz/columnLabels.csv").columns
for person_number in tqdm(range(1, 81 + 1)):


    csv_path = f"{part1_path}/{person_number}.csv/{person_number}.csv"
    if not os.path.exists(csv_path):
        csv_path = f"{part2_path}/{person_number}.csv/{person_number}.csv"
    df = pd.read_csv(csv_path,
                header=None,
                names=column_list)
    trials_list = set(df.trial)


    for t1, trial_number in enumerate(trials_list):
        number_of_trials = len(df[df.trial == trial_number])
        if number_of_trials == 9216.0:
            current_sample_matrix = df[df.trial == trial_number][electrodes_list].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            X[x_counter] = averaged_by_N_big_vec.astype(np.float32)
            Y[x_counter] = diagnosis_dict[person_number]
            x_counter += 1
print("Total trials with the Appropriate number of measurements - ", x_counter)
X = X[: x_counter]
Y = Y[: x_counter]

print("Total trials with the Appropriate number of measurements- ", x_counter)

  0%|          | 0/81 [00:00<?, ?it/s]

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y, test_size=0.2, shuffle=True, random_state=42)
_norm = X

X_train = X_train_norm.reshape(X_train_norm.shape[0], -1, len(electrodes_list))
X_test = X_test_norm.reshape(X_test_norm.shape[0], -1, len(electrodes_list))

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
def create_model(params):
    model = Sequential()
    
    model.add(LSTM(params['lstm_units'], input_shape=(X_train.shape[1], X_train.shape[2]), activation='tanh', return_sequences=True))
    model.add(Dropout(params['dropout_rate']))
    
    model.add(LSTM(params['lstm_units'], activation='tanh', return_sequences=True))
    model.add(Dropout(params['dropout_rate']))
    
    model.add(GRU(params['lstm_units'], activation='tanh'))
    model.add(Dropout(params['dropout_rate']))
    
    model.add(Dense(params['dense_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dropout(0.4))
    
    model.add(Dense(params['dense_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dropout(0.4))
    
    model.add(Dense(1, activation='sigmoid'))
    
    custom_optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])
    
    return model

# Define your objective function for hyperparameter optimization
def evaluate_hyperparameters(params):
    model = create_model(params)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2, batch_size=params['batch_size'], callbacks=[early_stopping])
    val_loss = history.history['val_loss'][-1]
    
    return val_loss

# Define the search space for hyperparameters
search_space = {
    'lstm_units': [128, 192, 256],
    'dropout_rate': [0.1, 0.2, 0.3],
    'dense_units': [128, 256, 512],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128]
}

# Split your data into training and validation sets
# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform random search for hyperparameter optimization
num_iterations = 20
best_params = None
best_val_loss = float('inf')

for iteration in range(num_iterations):
    params = {
        param_name: np.random.choice(param_values) for param_name, param_values in search_space.items()
    }
    
    val_loss = evaluate_hyperparameters(params)
    
    if val_loss < best_val_loss:
        best_params = params
        best_val_loss = val_loss

# Train the final model with the best hyperparameters
final_model = create_model(best_params)
final_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=best_params['batch_size'], callbacks=[early_stopping])


IndexError: tuple index out of range